# Pre requisites - INSTALL SDK + configure .env
- Install Azure ML SDK v1.55 - recommended option is A) 
    - Option A) via ESML conda: Use conda/kernel in ESML: `azure_automl_esml_v155`, or kernel: `azure_automl_esml_v155_v155` (supports both SDK v1.55 and SDK v2 v 1.15)
        - How to install: https://github.com/jostrm/azure-enterprise-scale-ml/blob/main/environment_setup/user_dev_env_install/01-install-azureml-sdk-v1+v2.md
    - Option B) via Microsoft docs. Then you also need to pip install `%pip install -U python-dotenv`
        - https://learn.microsoft.com/en-us/python/api/overview/azure/ml/install?view=azure-ml-py
- Create an .env file, with the 3 variables: 
    - AZURE_CLIENT_ID - value from project specific keuvault for secret: `esml-project001-sp-id`
    - AZURE_CLIENT_SECRET - value from project specific keuvault for secret: `esml-project001-sp-secret`
    - AZURE_TENANT_ID - value from project specific keuvault for secret: `esml-tenant-id`


# STATUS: Does NOT Work, since Azure ML SDK v2 
- Error: It cannot talk private networking, hence the last cell will fail, creation of Compute Instance
- Kernel/Conda: You need to use: `azure_automl_esml_v155_v155` (supports both SDK v1.55 and SDK v2 v 1.15), since SDK v2 in this notebook.
- Use notebook  `10-v1-setup-computeinstance.ipynb` instead
    - Which does same thing. But with Azure MLSDK v1

### Pip these 3 cells, or not - depends if you did INSTALL SDK
- If you have installed: "azure-enterprise-scale-ml\environment_setup\user_dev_env_install\AzureML_v1_55_and_v2_1_15"
    Then you dont need to run below PIP
- If you have installed: "azure-enterprise-scale-ml\environment_setup\user_dev_env_install\AzureML_v1_55"
    - Then you need to run below PIP (3 cells)
        - azure-ai-ml, azure-identity, python-dotenv


In [ ]:
%pip install azure-ai-ml

In [ ]:
%pip install azure-identity

In [ ]:
%pip install -U python-dotenv

In [ ]:
subscription_id="guid"
resource_group_name="name"
workspace_name="name"

# TODO: CONFIGURE THIS
project_number="1"
region_short = "weu"
aifactory_environment = "dev" # dev,test,prod
aifactory_prefix = "dc-heroes"

sku_array = ["Standard_DS11_v2","STANDARD_NC24RS_V3", "Standard_D13_v2"]
compute_instance_sku = sku_array[0]
compute_instance_suffix = "01"
# TODO: CONFIGURE THIS

# NO TOUCH: Auto, since convention:
ai_factory_index = "001"
resource_group_name=aifactory_prefix+"-esml-project00"+project_number+"-"+region_short+"-"+aifactory_environment+"-"+ai_factory_index+"-rg"
resource_group_name_common=aifactory_prefix+"-esml-common-"+region_short+"-"+aifactory_environment+"-"+ai_factory_index
workspace_name="aml-prj00"+project_number+"-"+region_short+"-"+aifactory_environment+"-"+ai_factory_index
vnet_name = "vnt-esmlcmn-"+region_short+"-"+aifactory_environment+"-"+ai_factory_index # vnt-esmlcmn-uks-dev-001
subnet_name = "snet-esml-cmn-001"
vnetId =  "/"+subscription_id+"/resourceGroups/"+resource_group_name_common+"/providers/Microsoft.Network/virtualNetworks/"+vnet_name+"/subnets/"
subnet_fully_qualified = vnetId+subnet_name

In [ ]:
print(resource_group_name)
print(resource_group_name_common)
print(workspace_name)
print(vnet_name)
print(subnet_fully_qualified)

# Option A) Default credential

In [ ]:
from azure.ai.ml import MLClient
from azure.identity import DefaultAzureCredential

ml_client = MLClient(
    DefaultAzureCredential(), subscription_id, resource_group_name, workspace_name
)

# Option B) Interactive login (forced)

In [ ]:
from azure.identity import DefaultAzureCredential, InteractiveBrowserCredential

credential = InteractiveBrowserCredential()

# Option C) Service Principal login - with client secret in .env file
- Create an .env file with the below properties: 

AZURE_CLIENT_ID=aGuid

AZURE_TENANT_ID=aGuid

AZURE_CLIENT_SECRET=aSecret



In [ ]:
import os
from dotenv import load_dotenv

print("Loading environment variables from .env file")
load_dotenv(".env")

from azure.identity import DefaultAzureCredential

credential = DefaultAzureCredential()
# Check if given credential can get token successfully.
credential.get_token("https://management.azure.com/.default")

# After option a,b or c the below LOGIN code can be used

In [ ]:
from azure.ai.ml import MLClient
try:
    ml_client = MLClient.from_config(credential=credential)
except Exception as ex:
    # NOTE: Update following workspace information to contain
    #       your subscription ID, resource group name, and workspace name
    client_config = {
        "subscription_id": subscription_id,
        "resource_group": resource_group_name,
        "workspace_name": workspace_name,
    }

    # write and reload from config file
    import json, os

    config_path = "../.azureml/config.json"
    os.makedirs(os.path.dirname(config_path), exist_ok=True)
    with open(config_path, "w") as fo:
        fo.write(json.dumps(client_config))
    ml_client = MLClient.from_config(credential=credential, path=config_path)

print(ml_client)

# COMPUTE INSTANCE: No vnet, since SDK v2, only in SDK v1 this works

## Workarounds
- In AML SDK v1, we can specify another resource group where the vNet resides.
- Using Azure CLI, we can borrow the SUBNET parameter, and pass fully qualified name, which will override vNet 
    - Name of the subnet. Can also reference a subnet in an existing vnet by ID instead of name. 
    - If subnet ID is specified then vnet-name will be ignored. Subnet ID can refer to a vnet/subnet in another RG by specifying the fully qualified subnet ID. Required when vnet name is specified.
(In SDK v2 the AML workspace and the vNet need to be in the same Resource group (which rarely is the case...)

In [ ]:
# Compute Instances need to have a unique name across the region.
# Here we create a unique name with current datetime
from azure.ai.ml.entities import ComputeInstance, AmlCompute
import datetime

ci_basic_name = "p"+project_number+"-m"+compute_instance_suffix+"-xyz-dev-ci"+compute_instance_suffix # + datetime.datetime.now().strftime("%Y%m%d%H%M")

ci_basic = ComputeInstance(name=ci_basic_name, size=compute_instance_sku)
ml_client.begin_create_or_update(ci_basic).result()

# Note that idle_time_before_shutdown has been deprecated.
ComputeInstance(name=ci_basic_name, size=compute_instance_sku, idle_time_before_shutdown_minutes="30")

# COMPUTE INSTANCE: vNet secure, since below is SDK v1m

In [ ]:
from azure.ai.ml.entities import ComputeInstance, AmlCompute, ComputeSchedules, ComputeStartStopSchedule, RecurrenceTrigger, RecurrencePattern
from azure.ai.ml.constants import TimeZone
import datetime
from azure.ai.ml.entities import NetworkSettings
from azure.ai.ml.entities import Schedule

# Compute Instances need to have a unique name across the region.
ci_basic_name = "p00"+project_number+"-m01-pggpu-dev-ci"+compute_instance_suffix # + datetime.datetime.now().strftime("%Y%m%d%H%M")

ci_minimal_name = "ci-shutdown-all-days"
ci_start_time = "2024-04-01T20:00:00" #specify your stop time in the format yyyy-mm-ddThh:mm:ss. interval=1 (every day) interval=2 (every other day)
rec_trigger = RecurrenceTrigger(start_time=ci_start_time, time_zone=TimeZone.CENTRAL_EUROPE_STANDARD_TIME, frequency="week", interval=1, schedule=RecurrencePattern(week_days=["monday","tuesday","wednesday","thursday","friday", "saturday", "sunday"], hours=15, minutes=[30]))
myschedule = ComputeStartStopSchedule(trigger=rec_trigger, action="stop")
com_sch = ComputeSchedules(compute_start_stop=[myschedule])

nw_settings = NetworkSettings(vnet_name=vnet_name, subnet=subnet_fully_qualified)

ci_basic = ComputeInstance(name=ci_basic_name, size=compute_instance_sku, ssh_public_access_enabled=False,network_settings=nw_settings,enable_node_public_ip=False, idle_time_before_shutdown_minutes=30, schedules=com_sch)
ml_client.begin_create_or_update(ci_basic).result()

# Note that idle_time_before_shutdown has been deprecated.
ComputeInstance(name=ci_basic_name, size=compute_instance_sku, idle_time_before_shutdown_minutes="30")